<a href="https://colab.research.google.com/github/Moly-malibu/Front_End/blob/master/Spotify_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2019_most_streamed_tracks_audio_features.csv to 2019_most_streamed_tracks_audio_features (1).csv
Saving data.csv to data (6).csv
Saving SpotifyAudioFeaturesApril2019.csv to SpotifyAudioFeaturesApril2019.csv


In [ ]:
df1 = pd.read_csv('2019_most_streamed_tracks_audio_features.csv')
df2 = pd.read_csv('data.csv')
df3 = pd.read_csv('SpotifyAudioFeaturesApril2019.csv')

In [ ]:
print(df1.shape, df2.shape, df3.shape)

(1717, 24) (2017, 17) (130663, 17)


In [ ]:
df = pd.concat([df1, df2, df3], ignore_index=True, sort=False)
print (df.shape)

(134397, 32)


In [ ]:
df.head()

,Country,Rank,Track_id,Streams,Track Name,Artist,URL,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,duration_ms,key,mode,Artist_id,Artist_popularity,Artist_follower,Artist_img,Unnamed: 0,target,song_title,artist,artist_name,track_id,track_name,popularity
0,global,1.0,25sgk305KZfyuqVBQIahim,1.166186e+09,Sweet but Psycho,Ava Max,https://open.spotify.com/track/25sgk305KZfyuqV...,0.0691,0.719,0.704,0.000000,0.1660,-4.724,0.0476,0.628,133.002,4.0,187436,1,1,4npEfmQ6YuiwW1GpUmaq3F,87.0,1025358.0,https://i.scdn.co/image/4b914c6470c8458674538a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,global,2.0,2Fxmhks0bxGSBdJ92vM42m,1.052359e+09,bad guy,Billie Eilish,https://open.spotify.com/track/2Fxmhks0bxGSBdJ...,0.3280,0.701,0.425,0.130000,0.1000,-10.965,0.3750,0.562,135.128,4.0,194088,7,1,6qqNVTkY8uBg9cP3Jd7DAH,98.0,18867904.0,https://i.scdn.co/image/2622edec99d68d1d141886...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,global,3.0,6ocbgoVGwYJhOv1GgI9NsF,7.890940e+08,7 rings,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,0.5920,0.778,0.317,0.000000,0.0881,-10.732,0.3340,0.327,140.048,4.0,178627,1,0,66CXWjxzNUsdJxJ2JdwvnR,97.0,40958727.0,https://i.scdn.co/image/b1dfbe843b0b9f54ab2e58...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,global,4.0,1rgnBhdG2JDFTbYkYRZAku,7.642083e+08,Dance Monkey,Tones and I,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,0.6880,0.825,0.593,0.000161,0.1700,-6.401,0.0988,0.540,98.078,4.0,209755,6,0,2NjfBq1NflQcKSeiDooVjY,92.0,484241.0,https://i.scdn.co/image/c128f5ef4d210a67610acd...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,global,5.0,6v3KW9xbzN5yKLt9YKDYA2,7.630644e+08,Señorita,Shawn Mendes,https://open.spotify.com/track/6v3KW9xbzN5yKLt...,0.0392,0.759,0.548,0.000000,0.0828,-6.049,0.0290,0.749,116.967,4.0,190800,9,0,7n2wHs1TKAczGzO7Dd2rGr,94.0,23152263.0,https://i.scdn.co/image/6bd59cfbd3e1e6394af710...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df.duplicated()]

,Country,Rank,Track_id,Streams,Track Name,Artist,URL,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,duration_ms,key,mode,Artist_id,Artist_popularity,Artist_follower,Artist_img,Unnamed: 0,target,song_title,artist,artist_name,track_id,track_name,popularity


In [ ]:
df_no_dup = df[~df.duplicated(keep='first')]

In [ ]:
df_reduced = df_no_dup.drop_duplicates(subset='track_id', keep='first')

In [ ]:
df_reduced.shape

(130327, 32)

In [ ]:
def save_data_frame_as_csv(df=None, file=None):
    """
    Save a Pandas DataFrame to CSV format.
    """
    try:
        if not file.endswith('.csv'):
            file += '.csv'
        df.to_csv(file, index=False)
        print(f'DataFrame saved as: {file}')
    except Exception as e:
        print("DataFrame couldn't be saved: ", sys.exc_info()[0])
        raise

In [ ]:
save_data_frame_as_csv(df_reduced, './merged_data.csv')

DataFrame saved as: ./merged_data.csv


In [ ]:
def save_data_frame_as_json(df=None, file=None, orient='records'):
  """
  Save a Pandas DataFrame to Json format.
  """
  try:
    if not file.endswith('.json'):
      file += '.json'
    df.to_json(file, orient=orient)
    print(f'Dataframe saved as: {file}')
  except Exception as e:
    print("Dataframe couldn't be saved: ", sys.exc_info()[0])
    raise

In [ ]:
save_data_frame_as_json(df_reduced, '.merged_data.json', orient='records')

Dataframe saved as: .merged_data.json


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="9dbbb1bd0fd447938c8627d25202f101" 
secret = "3180a0da2b72451c9002d4d688dd9529"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

In [ ]:
#EDA + DATA PREPARATION

print('number of elements in the track_id list:', len(track_id))

In [ ]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

In [ ]:
df_tracks.info()

In [ ]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

In [ ]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [ ]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

In [ ]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()